# Scrapper de sites

> Récupère le contenu de chacun des URL classé par typologie (variable ndf dans le code), chargé depuis le fichier Sites.json.

> Stocke le résultat dans ndf qui conditionne la variable "fichier de sortie". Celle-ci peut être adaptée pour pointer sur une zone correcte de votre drive dans la zone "Personnalisation" dénotée ci-dessous. Les données des sites collectés sont stockées au format pickle dans le dossier /OUT/ContenusSites classé par typologie selon Sites.json.

> La variable de sortie est de type dictionnaire pour conserver les url sources (flexibilité pour pouvoir corriger). Cf. commentaires infra. 




In [ ]:
# Il faut initialiser le dossier à partir de GitHub
# cette action est à faire une seule fois lors de la première utilisation
!git clone https://github.com/Patent2net/DistorsionCom.git

Cloning into 'DistorsionCom'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 251 (delta 86), reused 213 (delta 48), pack-reused 0
Receiving objects: 100% (251/251), 41.77 MiB | 23.55 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
stockageEntree = "/content/DistorsionCom/OUT"
stockageSortie = "/content/DistorsionCom/OUT"

In [ ]:
import os, urllib
os.chdir('DistorsionCom')
from outils import isPartner, text_from_html, myRequest

In [ ]:
html = urllib.request.urlopen('http://www.portcros-parcnational.fr/fr').read()
print(text_from_html(html))

    Aller au contenu principal                Toggle navigation       Le Parc national de Port-Cros  Le Parc national en bref   L'établissement public   La réglementation   La charte   Marchés publics   Offres d'emploi   Recueil des actes administratifs   Le magazine du Parc national            Toggle navigation      Les parcs nationaux de France Parc national de la Vanoise  Parc national de Port-Cros  Parc national des Pyrénées  Parc national des Cévennes  Parc national des Ecrins  Parc national du Mercantour  Parc national de la Guadeloupe  Parc amazonien de Guyane  Parc national de La Réunion  Parc national des Calanques  Parc national de Forêts  Portail des parcs nationaux de France  Office français de la biodiversité                                Parc national de Port-Cros et Porquerolles         Toggle navigation         Des découvertes Faites de rencontres & d’émotions   Une destination d'exception   Les trois îles  Le littoral  La mer et les fonds marins  Des découvertes en to


---
Personnalisation

> Ces cellules permettent de réaliser les traitements à partir de son propre espace de stockage. A n'exécuter que dans ce cas en adaptant les dossiers d'entrée et sortie. NE PAS EXECUTER SAUF A PERSONNALISER LES TRAITEMENTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Récupérer les entrées sur son drive
stockageEntree = "/content/drive/MyDrive/OUT"

In [ ]:
# Récupérer les sorties sur son drive (créer un dossier "OUT")
stockageSortie = "/content/drive/MyDrive/"

---

In [ ]:
# recup import requests, re, pickle
from bs4 import BeautifulSoup
from urllib import parse 
import pickle
import requests
import re


In [ ]:
with open (stockageEntree + "/Ressources/NIVEAU1-PNPC.csv", "r") as partners:
  donnee= partners.readlines()
partenaires = []
for lig in donnee[1:]:
  urls = lig.split(";")[2]
  partenaires.extend(urls.split())


In [ ]:
partenairesPropres = []

In [ ]:
for url in set(partenaires):
  url = url.strip()
  url = url.replace('"', "")

  urlP = parse.urlparse(url)
  partenairesPropres.append(urlP.scheme + '://' + urlP.hostname)

In [ ]:
print(len(set(partenairesPropres)))

356


> recup des déjà collectés

In [ ]:
fichierDeSortie = stockageEntree + '/ContenusSites/' + 'Partners.pkl'
with open (fichierDeSortie, 'rb') as  fictemp: #on met tous les contenus dans pickle
  listePartners = pickle.load(fictemp)

In [ ]:
!pip install requests

In [ ]:
import urllib3

urllib3.disable_warnings() # pourquoi les certificats SSL ne passent pas aujourd'hui ? 30/11 

# Première boucle pour lever les problèmes

> La dynamique du web fait que certains sites peuvent ne pas être accessibles, ne pas répondre à un instant t, ou faire planter le collecteur. Ce qui suit teste chaque URL et construit la variable BadUrl avec les urls en erreur.

In [ ]:
BadUrl = []
Done = []
for url in partenaires: 
  if url not in listePartners .keys() and isPartner(url, partenairesPropres+partenaires):
      try:
        webpage = myRequest(url)
      except:
        print("bad", url)
        BadUrl.append(url)

In [ ]:
print (len(BadUrl ))

33


In [ ]:
done = [] 
listePartners = dict()

In [ ]:
for url in partenaires: 
  webpage ="" 
  if url not in listePartners .keys() and isPartner(url, partenairesPropres+partenaires):
    if url not in BadUrl and url not in done:
      #soup = dict() # changement de type de données pour conserver l'URL source
  # récupère l'URL d'un site web et enregistre la page web
      del(webpage)
      try:
        webpage = myRequest(url)
      except:
        urlTemp = url
        url = url .replace('http:', "https:")
        try:
          webpage = myRequest(url)
        except:
          if 'www' not in url:
            url = url .replace('https://', "https://www.")
          try:
            webpage = myRequest(url)
          except:
            if 'pelagos' in urlTemp or urlTemp .endswith('.org/') or urlTemp .endswith('.org'):
              try :
                print('test de ', urlTemp +'/fr')
                webpage = myRequest(urlTemp +'/fr')
              except:
                continue
            else:
              continue
  # récupère le contenu de la page web à l'aide de BeautifulSoup
    #soup.append(BeautifulSoup(webpage.content, "html.parser"))
      tempoSoup = BeautifulSoup(webpage.content, "html.parser")
      if tempoSoup.title is not None:
        titre = tempoSoup.title.text 
      else:
        titre = ""
      if tempoSoup.description is not None:
        desc = tempoSoup.description.text 
      else:
        desc = ""
      texte = text_from_html(webpage.content)
      if len(titre)>0 and len(desc)>0:
        listePartners[url] = titre + '\n' + desc + "\n" + texte
      elif len(titre)>0:
        listePartners[url] = titre + "\n" + texte
      elif len(desc)>0:
        listePartners[url] = desc + "\n" + texte
      else:
        listePartners[url] = texte
      # le contenu est dans le "casier" nommé par l'url
    # nettoyage
    
    # soup [cle] = re.sub('^a-zA-Z0-9àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      re.sub('^a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', listePartners [url])
      listePartners[url] = listePartners[url].replace("\xa0", " ")
      listePartners[url] = listePartners[url].replace("\n", " ")
      listePartners[url] = listePartners[url].replace("’", "'")
      listePartners[url] = listePartners[url].translate('utf8')
      dictionary = {"\\": ""}
      transtable= listePartners[url].maketrans(dictionary)
      listePartners[url] = listePartners[url].translate(transtable)
      listePartners[url] = str(listePartners[url])
      
    # c'est pas tip top. Certains caractères restent. Variable selon les sites. 

    # sauvegarde sur   Google Drive
      fichierDeSortie = stockageSortie + '/OUT/ContenusSites/Partners.pkl'
      with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans un pickle
        pickle.dump(listePartners, fictemp)
    done .append (url)

In [ ]:
len(set(listePartners.values()))

108

## Commentaires et exemple de ce qui est récupéré


*   les clés du dictionnaire de sortie sont les url
*   Le contenu du dictionnaire pour une clé donne le texte récupéré
* L'équivalent avec le précédent format (toutes les pages dans un texte) s'obtient facilement



In [ ]:
listePartners [url]

"Accueil - Sauvegarde des forets varoises               A propos   Qui sommes-nous  Découvrir le projet COPAINS    Nos services   Espaces naturels  Espaces verts  Propreté urbaine  Nettoyage des plages    Nos produits bio   Tous nos produits  Où trouver nos produits ?  Le panier Porquerollais  Conditions de Livraison    Agenda  Devenir partenaire   Particuliers  Entreprises  Nos partenaires    Contact  0 Article           Sélectionner une page                             Sauvegarde Des Forêts Varoises Sauvegarde des Forêts Varoises,  acteur local de l'environnement au service de l'emploi et du développement des territoires depuis 1991.         Pour la préservation de l'environnement et l'inclusion dans l'emploi             Une Economie Sociale et Solidaire au service du  développement des territoires et de l'emploi   Nous sommes engagés sur des actions de préservation et de valorisation de notre environnement dans le cadre d'un projet solidaire qui soutient l'insertion par l'emploi.   